# Example 6: qudit-controlled-phase gate

![](CCP_gate.svg "Example")
Note that for layers that operate only on some modes, add the 'atten' identity component to let the program know that we still need these modes and do not want to discard them.

In [47]:
import json
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from json_netlist import instantiate_netlist_components, sp_calculate_effective_matrix
from two_photon_conversion import sp_construct_operator, sp_create_filter_matrix

In [48]:
netlist = """
{
    "bd12" : {
      "type" : "atten",
      "input_modes" : [0, 1, 2, 3, 4, 5],
      "output_modes" : [0, 1, 2, 3, 4, 7],
      "layer" : 0,
      "arguments" : [1, 1, 1, 1, 1, 1],
      "kw_args" : {"sympy" : false, "integer" : true}
    }, 
    "hwpx" : {
      "type" : "hwp",
      "input_modes" : [6, 7],
      "output_modes" : [6, 7],
      "layer" : 1,
      "arguments" : ["x", 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "hwp0" : {
      "type" : "hwp",
      "input_modes" : [4, 5],
      "output_modes" : [4, 5],
      "layer" : 1,
      "arguments" : [0, 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "propagation1" : {
      "type" : "atten",
      "input_modes" : [0, 1, 2, 3],
      "output_modes" : [0, 1, 2, 3],
      "layer" : 1,
      "arguments" : [1, 1, 1, 1],
      "kw_args" : {"sympy" : false, "integer" : true}
    }, 
    "propagation2" : {
      "type" : "atten",
      "input_modes" : [0, 1, 2, 3],
      "output_modes" : [0, 1, 2, 3],
      "layer" : 2,
      "arguments" : [1, 1, 1, 1],
      "kw_args" : {"sympy" : false, "integer" : true}
    },   
    "qwp0pt1" : {
      "type" : "qwp",
      "input_modes" : [4, 5],
      "output_modes" : [4, 5],
      "layer" : 2,
      "arguments" : [0, 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "qwp0pt2" : {
      "type" : "qwp",
      "input_modes" : [6, 7],
      "output_modes" : [6, 7],
      "layer" : 2,
      "arguments" : [0, 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "CentralBS_pt1" : {
      "type" : "BS2pol",
      "input_modes" : [2,3,6,7],
      "output_modes" : [2,3,6,7],
      "layer" : 3,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "CentralBS_pt2" : {
      "type" : "BS2pol",
      "input_modes" : [0,1,null,null],
      "output_modes" : [0,1,null,null],
      "layer" : 3,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "CentralBS_pt3" : {
      "type" : "BS2pol",
      "input_modes" : [null,null,4,5],
      "output_modes" : [null,null,4,5],
      "layer" : 3,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "hwp45pt1" : {
      "type" : "hwp",
      "input_modes" : [0, 1],
      "output_modes" : [0, 1],
      "layer" : 4,
      "arguments" : ["arg45", 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "hwp45pt2" : {
      "type" : "hwp",
      "input_modes" : [2, 3],
      "output_modes" : [2, 3],
      "layer" : 4,
      "arguments" : ["arg45", 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "hwp45pt3" : {
      "type" : "hwp",
      "input_modes" : [4, 5],
      "output_modes" : [4, 5],
      "layer" : 4,
      "arguments" : ["arg45", 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "hwp45pt4" : {
      "type" : "hwp",
      "input_modes" : [6, 7],
      "output_modes" : [6, 7],
      "layer" : 4,
      "arguments" : ["arg45", 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "AttenBS1_pt1" : {
      "type" : "BS2pol",
      "input_modes" : [0,1,null,null],
      "output_modes" : [0,1,null,null],
      "layer" : 5,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "AttenBS1_pt2" : {
      "type" : "BS2pol",
      "input_modes" : [2,3,null,null],
      "output_modes" : [2,3,null,null],
      "layer" : 5,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "AttenBS2_pt1" : {
      "type" : "BS2pol",
      "input_modes" : [4,5,null,null],
      "output_modes" : [4,5,null,null],
      "layer" : 5,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "AttenBS2_pt2" : {
      "type" : "BS2pol",
      "input_modes" : [6,7,null,null],
      "output_modes" : [6,7,null,null],
      "layer" : 5,
      "arguments" : ["Th","Tv"],
      "kw_args" : {"sympy" : true}
    },
    "hwpy" : {
      "type" : "hwp",
      "input_modes" : [6, 7],
      "output_modes" : [6, 7],
      "layer" : 6,
      "arguments" : ["y", 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "hwpz" : {
      "type" : "hwp",
      "input_modes" : [4, 5],
      "output_modes" : [4, 5],
      "layer" : 6,
      "arguments" : ["z", 0],
      "kw_args" : {"sympy" : true, "integer" : false}
    },
    "propagation3" : {
      "type" : "atten",
      "input_modes" : [0, 1, 2, 3],
      "output_modes" : [0, 1, 2, 3],
      "layer" : 6,
      "arguments" : [1, 1, 1, 1],
      "kw_args" : {"sympy" : false, "integer" : true}
    },
    "bd21" : {
      "type" : "atten",
      "input_modes" : [0, 1, 2, 3, 4, 5, 6, 7],
      "output_modes" : [0, 1, 2, 3, null, 5, 4, null],
      "layer" : 7,
      "arguments" : [1, 1, 1, 1, 1, 1, 1, 1],
      "kw_args" : {"sympy" : false, "integer" : true}
    }    
  }
"""
netlist = json.loads(netlist)
instances = instantiate_netlist_components(netlist)
ccpmap = sp_calculate_effective_matrix(instances)
ccpmap = sp.simplify(ccpmap.subs([('arg45', sp.pi/4)]))
ccpmap #print single photon map

Matrix([
[                0, sqrt(Th)*sqrt(Tv),                               0,                               0,                           0,                                                                              0],
[sqrt(Th)*sqrt(Tv),                 0,                               0,                               0,                           0,                                                                              0],
[                0,                 0,                               0,               sqrt(Th)*sqrt(Tv),                           0,                                              -I*sqrt(Th)*sqrt(1 - Tv)*cos(2*x)],
[                0,                 0,               sqrt(Th)*sqrt(Tv),                               0,                           0,                                                 sqrt(Tv)*sqrt(1 - Th)*sin(2*x)],
[                0,                 0, -sqrt(Tv)*sqrt(1 - Th)*sin(2*y), -sqrt(Th)*sqrt(1 - Tv)*cos(2*y),                           

In [49]:
allowed_inputs = [(i_signal, j_idler+4) for i_signal in range(4) for j_idler in range(2)]
allowed_outputs = [(i_signal, j_idler+4) for i_signal in range(4) for j_idler in range(2)]
n_map_rows = ccpmap.shape[0]
filter_matrix_11 = sp_create_filter_matrix(allowed_outputs, n_map_rows, True, True)
filter_matrix_01 = sp_create_filter_matrix(allowed_outputs, n_map_rows, False, True)
filter_matrix_10 = sp_create_filter_matrix(allowed_outputs, n_map_rows, True, False)
op_11 = sp_construct_operator(ccpmap, allowed_inputs, filter_matrix_11)
op_01 = sp_construct_operator(ccpmap, allowed_inputs, filter_matrix_01)
op_10 = sp_construct_operator(ccpmap, allowed_inputs, filter_matrix_10)


gate introduces pi/2 shift
but the success probability is lowered even more, from 1/9 to 1/18.

In [50]:
sigma_x = np.array([[0,1],[1,0]])
shift_pi = np.array([[1,0],[0,-1j]]) #shift of pi + phi/2
eye2 = np.array([[1,0],[0,1]])
flip = np.kron(np.kron(eye2, sigma_x), shift_pi @ sigma_x)
sp_flip = sp.Matrix(flip)

sub = [('Th', 1), ('Tv', 1/3), ('x', 0), ('y', 0), ('z', 0)]
mat_11 = sp_flip * op_11.subs(sub)
mat_01 = sp_flip * op_01.subs(sub)
mat_10 = sp_flip * op_10.subs(sub)
#sp.N(-mat, chop=1e-12)

In [51]:
import KetSugar as ks
import matplotlib.pyplot as plt
bellket = np.zeros((2**6, 1), dtype = complex)
for i in range(8):
    bellket += ks.BinKet((i << 3) + i, 2**6 - 1)

U_ideal = np.eye(8, dtype=complex)
U_ideal[-1, -1] = -1
chiket_ideal = np.kron(np.eye(8), U_ideal) @ bellket
chi_ideal = ks.ketbra(chiket_ideal, chiket_ideal)
chi_ideal = chi_ideal/np.trace(chi_ideal)

In [52]:
m0 = np.array(sp.N(-mat_11, chop=1e-12)).astype(complex)
m1 = np.array(sp.N(-mat_01, chop=1e-12)).astype(complex)
m2 = np.array(sp.N(-mat_10, chop=1e-12)).astype(complex)

chikets = [np.kron(np.eye(8), mi) @ bellket for mi in [m0, m1, m2]]
def make_rho(ket):
    m = ks.ketbra(ket, ket)
    return m/np.trace(m)

chis = [make_rho(chiket) for chiket in chikets]
q = 1
chi_eff = chis[0]*q + (1-q)*(chis[1]+chis[2])
print(np.round(ks.Fidelity(chi_eff, chi_ideal),5))

0.9929


In [53]:
std = 0*np.pi/180
intdev = np.sin(std)*0.5
eps = np.exp(-std**2 / 2)
e2 = np.eye(2)
deph = np.array([[1,eps],[eps,1]])
dephaser = ks.kron(deph, deph, deph, deph, deph, deph)

chi_deph = chi_eff * dephaser
print(intdev)
print(np.round(1-ks.Fidelity(chi_deph, chi_ideal),7)*100)


0.0
0.70961


to do in another version:
use various variables for BS transmittances and 
let them fluctuate around 3% and 0.3% and see histograms of process fidelity